In [8]:
## package imports ##
%load_ext autoreload
%autoreload 2
import sys
import pdkit
sys.path.append("../../src")
import utils.query_utils as query
import synapseclient as sc
import pandas as pd

syn = sc.login()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Welcome, aryton tediarjo!



INFO:synapseclient_default:Welcome, aryton tediarjo!



In [11]:
MPOWER_GAIT_DATA_V1 = "syn21111818"
MPOWER_DEMO_DATA_V1 = "syn10371840"
MPOWER_GAIT_DATA_V2 = "syn21113231"
MPOWER_DEMO_DATA_V2 = "syn15673379"
MPOWER_GAIT_DATA_PASSIVE = "syn21114136"
EMS_PROF_DATA = "syn10235463"
EMS_DEMO_DATA = "syn10295288"
EMS_GAIT_DATA = "syn21256442"
METADATA_COLS  = ['recordId', 'healthCode', 'appVersion', 
                    'phoneInfo', 'createdOn', 'PD', 'MS',
                    'gender', 'age', 'version']
GIT_URL = "https://github.com/arytontediarjo/mPower-Analysis/blob/master/src/clean.py"


In [12]:
def create_mPowerV1_interim_gait_data(GAIT_DATA, DEMO_DATA):
    """
    Function to format mpower version 1 data,
    list of formatting done:
        -> Clean table from test users
        -> Combine raw data with demographic table
        -> Fix column naming convention
        -> Map diagnosis to binary values
        -> Clean data that is below the range of 0-100
        -> Filter gender to male and female
    Parameters:
    GAIT_DATA = Takes in raw featurized gait data on version 1 (synapse file entity)
    DEMO_DATA = Takes in demographic data (synapse table entity)
    returns a formatized dataset of featurized gait data with its respective demographic data
    """
    demo_data = syn.tableQuery("SELECT age, healthCode, inferred_diagnosis as PD, gender FROM {} where dataGroups\
                               NOT LIKE '%test_user%'".format(DEMO_DATA)).asDataFrame()
    demo_data = demo_data[(demo_data["gender"] == "Female") | (demo_data["gender"] == "Male")]
    demo_data = demo_data.dropna(subset = ["PD"], thresh = 1)                     ## drop if no diagnosis
    demo_data["PD"] = demo_data["PD"].map({True :1.0, False:0.0})                 ## encode as numeric binary
    demo_data["age"] = demo_data["age"].apply(lambda x: float(x))                 ## convert age to float
    demo_data = demo_data[(demo_data["age"] <= 100) & (demo_data["age"] >= 10)]   ## subset to realistic age ranges
    demo_data["gender"] = demo_data["gender"].apply(lambda x: x.lower())          ## lowercase gender for consistencies
    gait_data = query.get_file_entity(syn = syn, synid = GAIT_DATA)
    data = pd.merge(gait_data, demo_data, on = "healthCode", how = "inner")
    data_return   = data[[feature for feature in data.columns if "outbound" not in feature]]
    data_outbound = data[[feature for feature in data.columns if "return" not in feature]]
    data = pd.concat([query.fix_column_name(data_outbound), query.fix_column_name(data_return)])## combine return and outbound                                                   
    data = data.reset_index(drop = True)
    data = data[[feat for feat in data.columns if ("." in feat) or (feat in METADATA_COLS)]]
    return data

In [13]:
dataV1                    = create_mPowerV1_interim_gait_data(GAIT_DATA = MPOWER_GAIT_DATA_V1, DEMO_DATA = MPOWER_DEMO_DATA_V1)
dataV1["version"]         = "mpower_v1"

[WARNING] /Users/atediarjo/Documents/SageBionetworks/synapse_venv/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (4,5,7,8,9,10,11,12,14,15,17,18,19,20,21,22,24,25,27,28,29,30,31,32,34,35,37,38,39,40,41,42,44,45,47,48,49,50,51,52,54,55,57,58,59,60,61,62,64,65,67,68,69,70,71,72,74,75,77,78,79,80,81,82) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)

  exec(code_obj, self.user_global_ns, self.user_ns)



In [15]:
dataV1[[feat for feat in dataV1.columns if "0" in feat]]

,AA.0,x.0,y.0,z.0
0,#ERROR,#ERROR,#ERROR,#ERROR
1,#ERROR,#ERROR,#ERROR,#ERROR
2,#ERROR,#ERROR,#ERROR,#ERROR
3,#ERROR,#ERROR,#ERROR,#ERROR
4,#ERROR,#ERROR,#ERROR,#ERROR
5,#ERROR,#ERROR,#ERROR,#ERROR
6,#ERROR,#ERROR,#ERROR,#ERROR
7,#ERROR,#ERROR,#ERROR,#ERROR
8,#ERROR,#ERROR,#ERROR,#ERROR
9,#ERROR,#ERROR,#ERROR,#ERROR
